# This notebook deals with site retrieval for soil moisture

In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob
import pandas as pd
from affine import Affine
from functools import partial
from multiprocessing import Pool
from osgeo import gdal

In [2]:
filePath = '/media/scratch/ZhiLi/SMAP/'
fnames= glob(filePath+'*.h5')

In [3]:
h5= h5py.File(np.random.choice(fnames),'r')

In [4]:
h5.keys()

<KeysViewHDF5 ['Metadata', 'Soil_Moisture_Retrieval_Data_AM', 'Soil_Moisture_Retrieval_Data_PM']>

In [5]:
h5Lons= h5['Soil_Moisture_Retrieval_Data_AM/longitude'][:]
h5Lats= h5['Soil_Moisture_Retrieval_Data_AM/latitude'][:]
h5Lons[h5Lons<-200]=np.nan
h5Lats[h5Lats<-200]=np.nan

In [7]:
h5Lons.shape

(1624, 3856)

In [6]:
siteDF= pd.read_excel('/home/ZhiLi/soilmoisture/Mesonet_Stations_info.xlsx')

## retrieve SMAP data (HDF5 basis)

In [10]:
def retrieval(df, fname):
    
    h5= h5py.File(fname, 'r')
    datetime= fname.split('/')[-1].split('_')[5]
#     smAct= h5['Soil_Moisture_Retrieval_Data_AM/soil_moisture'][:]
#     lonsAct= h5['Soil_Moisture_Retrieval_Data_AM/longitude'][:]
#     latsAct= h5['Soil_Moisture_Retrieval_Data_AM/latitude'][:]
    
    smPas= h5['Soil_Moisture_Retrieval_Data_PM/soil_moisture_pm'][:]
    lonsPas= h5['Soil_Moisture_Retrieval_Data_PM/longitude_pm'][:]
    latsPas= h5['Soil_Moisture_Retrieval_Data_PM/latitude_pm'][:]
    
    soilDF= pd.DataFrame()
    
    for i in range(len(df)):
        lon= df.elon.iloc[i]
        lat= df.nlat.iloc[i]
        
#         row, col= getIndex(lonsAct, latsAct, lon, lat)
        row, col= getIndex(lonsPas, latsPas, lon, lat)
        _soil= smPas[row, col]
#         if _soil<=-1:
#             row, col= getIndex(lonsPas, latsPas, lon, lat)
#             _soil= smPas[row, col]
            
        soilDF[df.stnm.iloc[i]]= _soil
        
    soilDF.index= [datetime]
    return soilDF
        
    

def getIndex(lons,lats, lon, lat):
    col= np.where(abs(lons[0,:] - lon)== np.nanmin(abs(lons[0,:] - lon)))[0]
    row= np.where(abs(lats[:,0] - lat) == np.nanmin(abs(lats[:, 0] - lat)))[0]
                  
    return row, col
    

In [11]:
thread= partial(retrieval, siteDF)

In [12]:
def main(files):
    pool= Pool(30)
    with Pool(30) as pool:
        results= pool.map(thread, files)
    df= pd.DataFrame()
    for result in results:
        df= pd.concat([df, result])
        
    return df

if __name__=='__main__':
    df= main(fnames)

In [207]:
df.to_csv('soilmoisture/SMAP_site.csv')

In [14]:
df.to_csv('SMAP_site_PM.csv')

## Retrieve NOAH model data (Grib basis)

In [8]:
import pygrib

In [9]:
fnames= glob('data/*.grb')

In [12]:
gribs= pygrib.open(fnames[0])
grib= gribs.message(4)
lats, lons= grib.latlons()

In [14]:
lats.shape

(27, 69)

In [10]:
pygrib.open(fnames[0]).select()

[1:86:86 (instant):regular_ll:depthBelowLandLayer:level None:fcst time 0 hrs:from 201504010000,
 2:250:250 (instant):regular_ll:depthBelowLandLayer:level None:fcst time 0 hrs:from 201504010000,
 3:86:86 (instant):regular_ll:depthBelowLandLayer:level None:fcst time 0 hrs:from 201504010000,
 4:86:86 (instant):regular_ll:depthBelowLandLayer:level None:fcst time 0 hrs:from 201504010000,
 5:86:86 (instant):regular_ll:depthBelowLandLayer:level None:fcst time 0 hrs:from 201504010000,
 6:86:86 (instant):regular_ll:depthBelowLandLayer:level None:fcst time 0 hrs:from 201504010000,
 7:86:86 (instant):regular_ll:depthBelowLandLayer:level None:fcst time 0 hrs:from 201504010000]

In [166]:
pygrib.open(fnames[0]).message(4).values.max()

33.900038719177246

In [167]:
def retrieve(df, fname):
    try:
        gribs= pygrib.open(fname)
        grib= gribs.message(4)
        lats, lons= grib.latlons()
        arr= grib.values
        soilDF= pd.DataFrame()
        gribs.close()
        datetime= fname.split('.')[1][1:]+fname.split('.')[2]

        for i in range(len(df)):
            lon= df.elon.iloc[i]
            lat= df.nlat.iloc[i]

            row, col= getIndex(lons, lats, lon, lat)

            _soil= arr[row, col]

            soilDF[df.stnm.iloc[i]]= _soil

        soilDF.index= [pd.to_datetime(datetime, format='%Y%m%d%H%M')]
        
        return soilDF
    
    except OSError:
        return fname
    

In [168]:
def main(files):
    global siteDF
    empty_files= []
    thread= partial(retrieve, siteDF)
    pool= Pool(35)
    with Pool(35) as pool:
        results= pool.map(thread, files)
    df= pd.DataFrame()
    for result in results:
        if isinstance(result, pd.DataFrame):
            df= pd.concat([df, result])
        else:
            empty_files.append(result)
        
    return (df, empty_files)

if __name__=='__main__':
    df, missingFiles= main(fnames)

In [170]:
df.to_csv('Noah_site.csv')

In [154]:
SMAP= pd.read_csv('SMAP_site.csv')

In [155]:
Noah= pd.read_csv('Noah_site.csv')

In [169]:
df.head()

,110,1,2,116,135,111,126,6,8,9,...,99,100,101,102,132,104,105,106,107,108
2015-04-01 00:00:00,16.610000,20.460097,21.880019,23.469863,20.819961,16.819961,19.830215,15.759902,20.409804,21.339980,...,21.810195,18.150039,21.139785,23.980117,20.440078,27.560195,23.980117,27.480117,16.139785,18.650039
2015-04-01 01:00:00,16.610000,20.469863,22.100234,23.469863,20.830215,16.819961,19.839980,15.770156,20.420058,21.350234,...,21.830215,18.159804,21.150039,23.989882,20.460097,27.589980,24.009902,27.509902,16.150039,18.659804
2015-04-01 02:00:00,16.619765,20.489882,29.580215,23.480117,20.839980,16.830215,19.850234,15.779922,20.420058,21.969863,...,21.839980,18.330215,21.170058,25.299941,20.500136,27.610000,24.029922,27.529922,16.159804,18.670058
2015-04-01 03:00:00,16.659804,20.520156,36.339980,23.480117,20.850234,16.909804,19.860000,15.779922,20.420058,28.949843,...,21.850234,20.279922,21.179824,32.909804,20.509902,27.630019,24.049941,27.549941,16.159804,18.679824
2015-04-01 04:00:00,19.300429,20.539687,35.159804,23.489882,20.860000,19.770156,19.879531,15.789687,20.419570,28.489882,...,21.940078,22.800429,21.260390,32.890273,20.539687,27.650039,24.069961,27.569961,16.169570,18.690078


In [157]:
SMAP.head()

,Unnamed: 0,110,1,2,116,135,111,126,6,8,...,99,100,101,102,132,104,105,106,107,108
0,20191201,0.223894,0.355768,0.153311,0.182192,0.380909,0.187627,0.296869,0.164766,0.165373,...,0.273996,0.192414,0.223894,0.182192,0.428555,0.464628,0.421116,0.461539,0.153311,0.358597
1,20150703,0.139557,0.223116,0.126063,0.126614,0.241755,0.139180,0.220274,0.133811,0.076322,...,0.144098,0.134915,0.139557,0.126614,0.264406,0.292729,0.210101,0.307210,0.126063,0.250518
2,20190515,0.299477,0.426432,0.183355,0.253759,0.367943,0.264975,0.373601,0.161254,0.162266,...,0.389624,0.266697,0.299477,0.253759,0.406057,0.344711,0.401537,0.329570,0.183355,0.394122
3,20190323,0.207803,0.289247,0.160307,0.195456,-9999.000000,0.204062,0.277960,0.132041,0.154042,...,0.222501,0.177907,0.282810,0.202363,-9999.000000,0.269637,-9999.000000,0.263933,0.141931,-9999.000000
4,20160813,0.216485,0.148873,0.211928,0.200590,-9999.000000,0.228069,0.156053,0.259724,0.354437,...,0.103026,0.200271,0.216485,0.200590,-9999.000000,-9999.000000,-9999.000000,-9999.000000,0.211928,-9999.000000


# grid SMAP data to raster

In [2]:
filePath = '/media/scratch/ZhiLi/SMAP/'
fnames= glob(filePath+'*.h5')

In [9]:
sorted(fnames)[-1]

'/media/scratch/ZhiLi/SMAP/SMAP_L3_SM_P_E_20200131_R16515_001.h5'

In [8]:
periods= pd.date_range('20150401', '20190702', freq='D')

In [10]:
def arr2raster(dst, arr, lons, lon_diff, lats, lat_diff):
    cols= arr.shape[1]
    rows= arr.shape[0]
    originX= lons[0]
    originY= lats[-1]
    driver= gdal.GetDriverByName('GTiff')
    outdata= driver.Create(dst, cols, rows, gdal.GDT_Float32)
    outdata.SetGeoTransform((originX, lon_diff, 0, originY, 0, -lat_diff))
    outdata.SetProjection('EPSG:4326')
    outdata.GetRasterBand(1).WriteArray(arr)

In [11]:
os.mkdir('/media/scratch/ZhiLi/SMAP_AM')

In [61]:
def grid(time, mode='AM'):
    fnameMatch= '/media/scratch/ZhiLi/SMAP/SMAP_L3_SM_P_E_%s_R16510_001.h5'%time.strftime('%Y%m%d')
    dst= '/media/scratch/ZhiLi/SMAP_AM/%s.tif'%time.strftime('%Y%m%d')
    with h5py.File(fnameMatch,'r') as h5:
        if mode=='AM':
            lons= h5['Soil_Moisture_Retrieval_Data_AM/soil_moisture'][:]
            lats= h5['Soil_Moisture_Retrieval_Data_AM/longitude'][:]
            soil= h5['Soil_Moisture_Retrieval_Data_PM/latitude_pm'][:]
        elif mode=='PM':
            lons= h5['Soil_Moisture_Retrieval_Data_PM/longitude_pm'][:]
            lats= h5['Soil_Moisture_Retrieval_Data_PM/latitude_pm'][:]
            soil= h5['Soil_Moisture_Retrieval_Data_PM/soil_moisture_pm'][:]
    return lons, lats, soil
    lon_diff= lons[0,0]- lons[0,1]
    lat_diff= lats[0,0]- lats[1,0]
    arr2raster(dst, soil, lons[0,:], lon_diff, lats[:,0], lat_diff)
#     os.system('gdal_translate -tr 0.125 -0.125 -projwin -103 37 -94.375 33.625 -r nearst %s %s'%(dst, dst))

In [62]:
lons, lats, soil= grid(periods[0])

In [32]:
periods[0]

Timestamp('2015-04-01 00:00:00', freq='D')

In [54]:
raster= gdal.Open('/media/scratch/ZhiLi/SMAP_AM/20150401.tif')

In [55]:
arr= raster.ReadAsArray()

In [56]:
arr.shape

(6, 1624, 3856)

In [57]:
raster.GetGeoTransform()

(-9999.0, 0.0, 0.0, -179.9533233642578, 0.0, -0.0)